# Merge HGHI and CCM data

HGHI is published at the HRR and State level. Merge this with the data generated by this project. This also includes ventilator information merged into HGHI at the state level.

In [ ]:
import pandas as pd
import geopandas as gpd

from covidcaremap.data import processed_data_path, published_data_path

In [ ]:
hghi_state_gdf = gpd.read_file(processed_data_path('hghi_state_data_with_vents.geojson'), 
                               encoding='utf-8')
hghi_hrr_gdf = gpd.read_file(processed_data_path('hghi_hrr_data.geojson'), 
                             encoding='utf-8')

ccm_hrr_gdf = gpd.read_file(published_data_path(
    'us_healthcare_capacity-hrr-CovidCareMap.geojson'), encoding='utf-8')
ccm_state_gdf = gpd.read_file(published_data_path(
    'us_healthcare_capacity-state-CovidCareMap.geojson'), encoding='utf-8')


### Merge by HRR

In [ ]:
hghi_hrr_prefixed_gdf = hghi_hrr_gdf.set_index('HRRCITY') \
                                    .drop(columns=['HRR', 'HRR_BDRY_I', 'HRRNUM', 'geometry']) \
                                    .add_prefix('HGHI - ')

In [ ]:
merged_hrr_gdf = ccm_hrr_gdf.set_index('HRRCITY').join(hghi_hrr_prefixed_gdf).reset_index()

In [ ]:
merged_hrr_gdf.to_file(processed_data_path('hghi_ccm_data_by_hrr.geojson'), 
                     driver='GeoJSON')

## Merge by State

In [ ]:
hghi_state_prefixed_gdf = hghi_state_gdf.set_index('State') \
                                        .drop(columns=['geometry']) \
                                        .add_prefix('HGHI - ')

In [ ]:
merged_state_gdf = ccm_state_gdf.set_index('State').join(hghi_state_prefixed_gdf).reset_index()


In [ ]:
merged_state_gdf.to_file(processed_data_path('hghi_ccm_data_by_state.geojson'), 
                         driver='GeoJSON')

In [ ]:
merged_state_gdf[merged_state_gdf['State'] == 'KY']